# Import

In [1]:
import sys
%matplotlib inline
SCRIPTS_PATH = '../scripts'
if SCRIPTS_PATH not in sys.path:
    sys.path.append(SCRIPTS_PATH)

import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import regression_model
import NNModel
from tagging import *
from NNModel import *
from window_aggregation import *
from dull_model import *
from collocation import *

Using TensorFlow backend.


# Load test data

In [2]:
path_to_data = '../data/test.csv'
test_df = pd.read_csv(path_to_data)

In [3]:
#full_df = pd.read_csv('../data/dataset1_v2.csv')

In [4]:
#prods = full_df['PRODUCT'].unique().tolist()
#selected_prods = np.random.choice(prods, 30)
#out_df = full_df[full_df['PRODUCT'].isin(selected_prods)]
#out_df.to_csv('../data/test.csv', index=False)

# Init models

In [5]:
nn_model = nn_init('../model/')

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.18.1 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
tag_model = tag_init('../model/dn/')

In [7]:
dull_model = DullModel('../model/rm/')

# Predict

In [1]:
products = test_df['PRODUCT'].unique()
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '
for product in products:
    comments = test_df[test_df['PRODUCT'] == product]['TEXT'].tolist()
    prod_info = test_df[test_df['PRODUCT'] == product].head(1)
    prod_name = prod_info['NAME'].tolist()[0].strip() if len(prod_info['NAME']) > 0 else ''
    print('Продукт %d, %s' % (product, prod_name))
    
    # DN - tags
    num_chosen_tags = 3
    single_prod_df = test_df[test_df['PRODUCT'] == product]
    tags = tag_model.predict(single_prod_df, top_n=num_chosen_tags)
    tags = [t for t in tags if len(t.split(' ')) <=3]
    tag_line = ''
    for tag in tags:
        tag_flt = ''.join([c for c in tag.lower().strip() if c in alphabet])
        tag_flt = tag_flt[1:len(tag_flt)] if tag_flt[0] == ' ' else tag_flt
        tag_line += '#%s ' % tag_flt.replace(' ', '_')
    print('\t' + tag_line)
    
    # coallocation tags
    num_chosen_coallocations = 3
    col_model = Collocation_find('../model/mk/stopw-1.csv', path_to_data)
    tags = col_model.predict(product)[:num_chosen_coallocations]
    tag_line = ''
    for tag in tags:
        tag_flt = ''.join([c for c in tag.lower().strip() if c in alphabet])
        tag_flt = tag_flt[1:len(tag_flt)] if tag_flt[0] == ' ' else tag_flt
        tag_line += '#%s ' % tag_flt.replace(' ', '_')
    print('\t' + tag_line)
    print(' ')
    
    # nn 
    wnd_agg = aggregate(comments, num_comments=3, model=nn_model, wnd_l=12, stride=6, positive_q=(0.8, 0.95), negative_threshold=3)
    print('\tПокупатели отмечают такие положительные моменты: ')
    if len(wnd_agg['+']) == 0:
        print('\t\t---')
    else:
        for com in wnd_agg['+']:
            print('\t\t...%s...' % com)
    print(' ')
            
    print('\tПокупатели отмечают такие отрицательные моменты: ')
    if len(wnd_agg['-']) == 0:
        print('\t\t---')
    else:
        for com in wnd_agg['-']:
            print('\t\t...%s...' % com)
    print(' ')
    
    # comments
    num_chosen_comments = 3
    dull_pres = dull_model.predict(single_prod_df, list(range(len(single_prod_df))))[:num_chosen_comments]
    print('\tПокупатели также находят важным: ')
    if len(dull_pres) == 0:
        print('\t\t---')
    else:
        for com in dull_pres:
            print('\t\t...%s...' % com)
    
    print(' ')        
    print(''.join(['*'] * 110))
    print(' ')

NameError: name 'test_df' is not defined